In [5]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

In [6]:
sagemaker.get_execution_role()

'arn:aws:iam::939595455984:role/kiarash-sagemaker'

In [7]:
es = Estimator(
    role=sagemaker.get_execution_role(),
    image_uri='939595455984.dkr.ecr.eu-west-1.amazonaws.com/dgl-citation-network:custom-torch-1.8',
    instance_type='ml.m5.large',
    instance_count=1,
    hyperparameters={
        'epochs': 100,
        'learning-rate': 0.01
    }
)

In [8]:
es.fit()

2021-03-26 12:15:39 Starting - Starting the training job...
2021-03-26 12:16:02 Starting - Launching requested ML instancesProfilerReport-1616760939: InProgress
......
2021-03-26 12:17:04 Starting - Preparing the instances for training......
2021-03-26 12:18:03 Downloading - Downloading input data
2021-03-26 12:18:03 Training - Downloading the training image.........
2021-03-26 12:19:23 Training - Training image download completed. Training in progress.DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch
/usr/local/lib/python3.9/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/cora_full
Epoch

In [13]:
es_predictor = es.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1
)

-------------!

In [11]:
test_samples = ['0']

# es_predictor.content_type = 'text/csv' 
es_predictor.serializer = sagemaker.serializers.CSVSerializer()

response = es_predictor.predict(test_samples, initial_args={'ContentType': 'text/csv'}) 
print(response)

b'tensor([[  4.6271,   3.1328,  -0.7734,   2.4674,   2.0575,  -0.0761,  -4.3208,\n          -2.1375,   3.4420,   0.4862,   1.6299,   5.0871,   5.3637,  -1.2500,\n          -5.0728,  -1.8494,  -1.9123,  -7.8683,  -7.2599,  -7.3289,   1.4372,\n          -3.3154,  -2.0654,  -2.2093,  -0.9716,  -0.6761,  -2.1878,  -5.8438,\n           0.4813,  -1.3457,   0.7249, -12.6670,  -4.8841,   1.1072,   7.6540,\n          -2.8031,  -1.5386,  -8.0475,   3.4275,  -0.9210,  -6.0783,  -3.6843,\n           3.9666,   3.2181,  -0.6135,   1.2106,   0.2229,  -0.2590,   9.0986,\n          -5.7604,  -5.2870,  -2.7012,   2.2441,   2.6202,   4.1308,   1.3312,\n          -0.0942,  -3.1558,  -0.1163,   2.0456,   1.9385,  -5.6722,  -6.4538,\n          -2.1106,  -5.4159,  -5.8349,  -0.5502,  -0.8306,  -0.8714,  -3.9067]],\n       grad_fn=<IndexBackward>)'


In [24]:
es_predictor.delete_endpoint()

## Batch Transform

In [18]:
sess = sagemaker.Session()
prefix = 'kiarash/dgl-citation-network'

In [23]:
batch_input = sess.upload_data(path='../data.csv', key_prefix=prefix + '/batch')

In [25]:
es_transformer = es.transformer( 
    instance_count=1, 
    instance_type='ml.m5.large'
)

In [27]:
es_transformer.transform(
    batch_input, 
    content_type='text/csv', 
    wait=True, 
    logs=True
)

..........................DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [26/Mar/2021 14:23:44] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [26/Mar/2021 14:23:44] "#033GET /execution-parameters HTTP/1.1#033" 404 -
[2021-03-26 14:23:44,166] ERROR in app: Exception on /invocations [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatc

UnexpectedStatusException: Error for Transform job dgl-citation-network-2021-03-26-14-19-27-147: Failed. Reason: AlgorithmError: See job logs for more information

In [ ]:
print(es_transformer.output_path)